In [26]:
using TyDeepLearning
using Random
using TyPlot
using TyImages

In [27]:
path = dataset_dir("digit")

"C:/APP/Mworks/Syslab 2024a/Library/artifacts/8d33d9a5e642fdd4d2eac97e303159f1eea1ee9a"

In [28]:
X, Y = DigitDatasetTrainData(path)
# valX, valY = DigitDatasetTestData(path)

([0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; … ;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; … ;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; … ;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;;; … ;;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; … ;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; … ;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00;;;; 0x00; 0x00; … ; 0x00; 0x00;;; 0x00; 0x00; … ; 0x00; 0x00

In [29]:
size(X), size(Y)

((5000, 1, 28, 28), (5000,))

In [43]:
# 拆分数据集 8/2
split = 0.8
num_train = Int32(round(size(X)[1]*split))
# p = randperm(5000)
# index1 = p[1:num_train]
# index2 = p[num_train+1:end]
# X_train  = X[index1, :, :, :]
X_train,X_test  = X[1:num_train,:,:,:], X[num_train+1:end, :, :, :]
Y_train,Y_test  = Y[1:num_train],Y[num_train+1:end]

([3, 3, 7, 7, 7, 7, 9, 9, 7, 7  …  1, 0, 5, 5, 4, 8, 2, 0, 2, 3], [9, 4, 1, 4, 0, 9, 2, 9, 2, 1  …  2, 2, 3, 9, 9, 8, 9, 8, 9, 3])

In [44]:
p2 = randperm(num_train)
index = p2[1:20]
figure(1)
for i in eachindex(range(1, 20))
    subplot(4, 5, i)
    imshow(X_train[index[i], 1, :, :])
end

In [45]:
imageAugmenter = imageDataAugmenter(;RandomRotation = 20)
X_train2 = permutedims(X_train, (1, 3, 4, 2))
imagesize = (28, 28)
augimds = augmentedImageDatastore(imagesize, X_train2, Y_train, imageAugmenter)

augimds = permutedims(augimds, (1, 4, 2, 3))
figure(2)
for i in eachindex(range(1, 20))
    subplot(4, 5, i)
    imshow(augimds[index[i], 1, :, :])
end

In [76]:
options = trainingOptions("CrossEntropyLoss", "Adam", "Accuracy", 1024, 20, 0.001; Shuffle =true , Plots = true,ExecutionEnvironment="CPU")
# options = trainingOptions("CrossEntropyLoss", "Adam", "Accuracy", 512, 20, 0.001; Shuffle =true , Plots = true,CheckpointPath="./checkpoint",ExecutionEnvironment="GPU")
# options = trainingOptions("CrossEntropyLoss", "Adam", "Accuracy", 512, 20, 0.001; Shuffle =true , Plots = true,CheckpointPath="./checkpoint",ExecutionEnvironment="auto")

Dict{Any, Any} with 14 entries:
  "metrics"              => "Accuracy"
  "optimizer"            => "Adam"
  "epoch"                => 20
  "batch_size"           => 1024
  "learning_rate"        => 0.001
  "verbose"              => true
  "momentum"             => 0.9
  "eval_data"            => nothing
  "loss"                 => "CrossEntropyLoss"
  "eval_label"           => nothing
  "plots"                => true
  "executionenvironment" => "CPU"
  "checkpointpath"       => nothing
  "shuffle"              => true

In [77]:
layers = SequentialCell([
    convolution2dLayer(size(X_train)[2], 8, 3),
    batchNormalization2dLayer(8),
    reluLayer(),
    maxPooling2dLayer(2; Stride = 2),
    convolution2dLayer(8, 16, 3),
    batchNormalization2dLayer(16),
    reluLayer(),
    maxPooling2dLayer(2; Stride = 2),
    convolution2dLayer(16, 32, 3),
    batchNormalization2dLayer(32),
    reluLayer(),
    flattenLayer(),
    fullyConnectedLayer(32 * 7 * 7, 10),
    softmaxLayer()]
)

PyObject SequentialCell<
  (0): Conv2d<input_channels=1, output_channels=8, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
  (1): BatchNorm2d<num_features=8, eps=1e-05, momentum=0.9, gamma=Parameter (name=1.gamma, shape=(8,), dtype=Float32, requires_grad=True), beta=Parameter (name=1.beta, shape=(8,), dtype=Float32, requires_grad=True), moving_mean=Parameter (name=1.moving_mean, shape=(8,), dtype=Float32, requires_grad=False), moving_variance=Parameter (name=1.moving_variance, shape=(8,), dtype=Float32, requires_grad=False)>
  (2): ReLU<>
  (3): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
  (4): Conv2d<input_channels=8, output_channels=16, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
  (5): BatchNorm2d<num_features=16, eps=1e-05, momentum=0.9, gamma=Parameter (name=5.g

In [78]:
net = trainNetwork(X_train, Y_train, layers, options)

None
Epoch:[  0/ 20], step:[    1/    4], loss:[2.308/2.308], time:2354.761 ms, lr:0.00100
Epoch:[  0/ 20], step:[    2/    4], loss:[2.287/2.298], time:593.011 ms, lr:0.00100
Epoch:[  0/ 20], step:[    3/    4], loss:[2.265/2.287], time:524.204 ms, lr:0.00100
Epoch:[  0/ 20], step:[    4/    4], loss:[2.238/2.275], time:2065.242 ms, lr:0.00100
Epoch time: 5607.018 ms, per step time: 1401.754 ms, avg loss: 2.275
Epoch:[  1/ 20], step:[    1/    4], loss:[2.210/2.210], time:478.268 ms, lr:0.00100
Epoch:[  1/ 20], step:[    2/    4], loss:[2.180/2.195], time:496.102 ms, lr:0.00100
Epoch:[  1/ 20], step:[    3/    4], loss:[2.148/2.179], time:359.230 ms, lr:0.00100
Epoch:[  1/ 20], step:[    4/    4], loss:[2.130/2.167], time:455.690 ms, lr:0.00100
Epoch time: 1804.377 ms, per step time: 451.094 ms, avg loss: 2.167
Epoch:[  2/ 20], step:[    1/    4], loss:[2.091/2.091], time:407.202 ms, lr:0.00100
Epoch:[  2/ 20], step:[    2/    4], loss:[2.050/2.070], time:367.674 ms, lr:0.00100
Epoch:

PyObject <mindspore.train.model.Model object at 0x0000022C119578C8>

In [79]:
save(layers, "./checkpoint/v1.ckpt")

In [81]:
net = TyDeepLearning.loadnetwork("./checkpoint/v1.ckpt", net=layers)

PyObject SequentialCell<
  (0): Conv2d<input_channels=1, output_channels=8, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
  (1): BatchNorm2d<num_features=8, eps=1e-05, momentum=0.9, gamma=Parameter (name=1.gamma, shape=(8,), dtype=Float32, requires_grad=True), beta=Parameter (name=1.beta, shape=(8,), dtype=Float32, requires_grad=True), moving_mean=Parameter (name=1.moving_mean, shape=(8,), dtype=Float32, requires_grad=False), moving_variance=Parameter (name=1.moving_variance, shape=(8,), dtype=Float32, requires_grad=False)>
  (2): ReLU<>
  (3): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
  (4): Conv2d<input_channels=8, output_channels=16, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
  (5): BatchNorm2d<num_features=16, eps=1e-05, momentum=0.9, gamma=Parameter (name=5.g

In [93]:
YPred = TyDeepLearning.predict(net, X_test)
# Int32(size(X)[1] - num_train)
Y_val = reshape(Y_test, (Int32(size(X)[1] - num_train)))
acc = accuracy(YPred, Y_val)

0.98